In [2]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

# Generate data paths with labels
def define_paths(data_dir):
    filepaths, labels = [], []
    folds = os.listdir(data_dir)
    for fold in folds:
        foldpath = os.path.join(data_dir, fold)
        for file in os.listdir(foldpath):
            filepaths.append(os.path.join(foldpath, file))
            labels.append(fold)
    return filepaths, labels

# Create dataframe from file paths and labels
def create_df(data_dir):
    files, classes = define_paths(data_dir)
    df = pd.DataFrame({'filepaths': files, 'labels': classes})
    train_df, dummy_df = train_test_split(df, train_size=0.8, shuffle=True, random_state=123, stratify=df['labels'])
    valid_df, test_df = train_test_split(dummy_df, train_size=0.5, shuffle=True, random_state=123, stratify=dummy_df['labels'])
    return train_df, valid_df, test_df

# CNN model with standard, dilated, and transpose convolutions
class BrainTumorCNN(nn.Module):
    def __init__(self, conv_type='standard'):
        super(BrainTumorCNN, self).__init__()
        if conv_type == 'standard':
            self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        elif conv_type == 'dilated':
            self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=2, dilation=2)
        elif conv_type == 'transpose':
            self.conv1 = nn.ConvTranspose2d(3, 16, kernel_size=3, stride=1, padding=1)
        
        self.conv_layers = nn.Sequential(
            self.conv1,
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        return self.fc_layers(x)

# Define data transformations and dataset
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.,), (1.,))
])

data_dir = '/kaggle/input/brain-tumor'
train_df, valid_df, test_df = create_df(data_dir)
dataset = ImageFolder(data_dir, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Train and evaluate the model
def train_and_evaluate(model, num_epochs=30):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        
        model.eval()
        val_loss, correct = 0.0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == labels).sum().item()

        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        accuracy = correct / len(val_loader.dataset)
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Accuracy: {accuracy * 100:.2f}%")
    
    return accuracy

# Train models with different convolution types
standard_model = BrainTumorCNN(conv_type='standard')
dilated_model = BrainTumorCNN(conv_type='dilated')
transpose_model = BrainTumorCNN(conv_type='transpose')

print("Training standard convolution model...")
standard_acc = train_and_evaluate(standard_model)

print("\nTraining dilated convolution model...")
dilated_acc = train_and_evaluate(dilated_model)

print("\nTraining transpose convolution model...")
transpose_acc = train_and_evaluate(transpose_model)

# Compare results
print(f"\nResults Comparison:")
print(f"Standard Convolution Accuracy: {standard_acc * 100:.2f}%")
print(f"Dilated Convolution Accuracy: {dilated_acc * 100:.2f}%")
print(f"Transpose Convolution Accuracy: {transpose_acc * 100:.2f}%")


Training standard convolution model...


Epoch [1/30]:   0%|          | 0/77 [00:00<?, ?it/s]

Train Loss: 0.7620 | Val Loss: 0.6439 | Accuracy: 71.62%


Epoch [2/30]:   0%|          | 0/77 [00:00<?, ?it/s]

Train Loss: 0.5330 | Val Loss: 0.5522 | Accuracy: 76.84%


...

Train Loss: 0.0031 | Val Loss: 0.2893 | Accuracy: 94.13%


Epoch [29/30]:   0%|          | 0/77 [00:00<?, ?it/s]

Train Loss: 0.0015 | Val Loss: 0.2968 | Accuracy: 94.13%


Epoch [30/30]:   0%|          | 0/77 [00:00<?, ?it/s]

Train Loss: 0.0009 | Val Loss: 0.3105 | Accuracy: 94.13%

Results Comparison:
Standard Convolution Accuracy: 93.80%
Dilated Convolution Accuracy: 94.29%
Transpose Convolution Accuracy: 94.13%
